<a href="https://colab.research.google.com/github/lihaoranIcefire/Portfolio/blob/main/TaxCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re

In [125]:
!wget -c https://raw.githubusercontent.com/lihaoranIcefire/Portfolio/refs/heads/main/TaxBrackets.csv

--2024-12-18 20:26:01--  https://raw.githubusercontent.com/lihaoranIcefire/Portfolio/refs/heads/main/TaxBrackets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 811 [text/plain]
Saving to: ‘TaxBrackets.csv’

TaxBrackets.csv     100%[===================>]     811  --.-KB/s    in 0s      

2024-12-18 20:26:01 (53.3 MB/s) - ‘TaxBrackets.csv’ saved [811/811]



In [119]:
dc = {
    "federal_tax_bracket":[
        [
            0,
            6400,
            18325,
            54875,
            109750,
            203700,
            256925,
            632750
        ],
        [
            0.0,
            0.1,
            0.12,
            0.22,
            0.24,
            0.32,
            0.35,
            0.37
        ]
    ],
    "capital_gain_tax_bracket":[
        [
            0,
            44625,
            492300
        ],
        [
            0.0,
            0.15,
            0.20
        ]
    ],
    "standard_deduction": 14600
}

In [122]:
with open("test.json", "w") as json_file:
    json.dump(dc, json_file, indent=4)

# Tax Calculator

In [111]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
import json

class TaxCalculator:
    """
    Personal tax calculator
    """

    def __init__(self,
                 status='single',
                 year=2024,
                 salary=175000,
                 bonus=35000,
                 short_term_capital_gain=0,
                 long_term_capital_gain=0,
                 tax_treaty=5000,
                 traditional_401k=0,
                 hsa=0,
                 donation=0,
                 federal_tax_withheld=0):

        self.status = status
        self.year = year
        self.salary = salary
        self.bonus = bonus
        self.short_term_capital_gain = short_term_capital_gain
        self.long_term_capital_gain = long_term_capital_gain
        self.traditional_401k = traditional_401k
        self.hsa = hsa
        self.donation = donation
        self.federal_tax_withheld = federal_tax_withheld

        with open(f"{status}_{year}_tax_info.json", "r") as file:
            tax_info = json.load(file)
        self.federal_tax_bracket = self.info["federal_tax_bracket"]
        self.capital_gain_tax_bracket = self.info["capital_gain_tax_bracket"]
        self.standar_deduction = self.info["standar_deduction"]

    @staticmethod
    def tax_liability(interval, rates, income, staircase=True):
        assert len(interval) == len(rates)

        if not staircase:
            for i in range(len(rates)-1):
                if interval[i] < income <= interval[i+1]:
                    return income * rates[i]

        accrued_sum = 0
        for i in range(len(rates)-1):
            if income < interval[i+1]:
                return accrued_sum + (income - interval[i]) * rates[i]
            else:
                accrued_sum += (interval[i+1] - interval[i]) * rates[i]
        return accrued_sum + (income - interval[-1]) * rates[-1]

    def federal_tax(self, deduction_method='standard'):

        if deduction_method == 'standard':
            taxable_income = self.salary \
                        + self.bonus \
                        + self.short_term_capital_gain \
                        - self.traditional_401k \
                        - self.hsa \
                        - self.standar_deduction

        return TaxCalculator.tax_liability(*self.federal_tax_bracket, taxable_income) \
             + TaxCalculator.tax_liability(*self.capital_gain_tax_bracket, self.long_term_capital_gain) \


    # def fica(icome)

    # def state_tax

    # def local_tax

    # def f1040

## Federal income tax

1. `annual_wage`: (paycheck's gross pay) $\times$ (pay frequency). The pay frequency is
    - once per quarter: **Quarterly = 4**
    - once a month: **Monthly = 12**
    - twice per month: **Semi-monthly = 24**
    - once every other week: **Bi-weekly = 26**
    - every week: **Weekly = 52**
2. `filing_status`: Either "single", "married filing separately", "married filing jointly", or "head of household".
3. `other_income`: The amount from Step 4(a) of Form W-4 (additional income not from jobs. Such as interests, dividends, retirement, etc.) and add it to the wages from Step 3. Write this result down.
4. `additional_deduction`: The amount from Step 4(b) of Form W-4 (additional deductions beyond the standard deduction).
5. `multiple_jobs`: The Step 2 box of Form W-4.
7. `adjusted_annual_wage`: Taxable income for the tax brackets.
8. Tax brackets are on page 11 of the IRS Publication 15T:
    - If the Step 2 box of Form W-4 is **not checked**, use the table on the left.
    - If the Step 2 box of Form W-4 is **checked**, use the table on the right.
9. `additional_credit`: Step 3 of Form W-4.
10. `additional_deduction`: Step 4(c) of Form W-4.

In [39]:
TaxCalculator.federal_income_tax(280000, filing_status="married filing jointly")

48790.0

In [40]:
TaxCalculator.federal_income_tax(280000)

65307.25

In [20]:
TaxCalculator.federal_income_tax(210000 - 23000 - 4150) / (210000)

0.1675952380952381

In [15]:
210000 - 23000 - 4150

182850

In [23]:
TaxCalculator.tax_liability([0, 20000, 35000, 40000, 75000, 500000, 1000000], [0.014, 0.0175, 0.035, 0.0525, 0.0637, 0.0897, 0.1075], 210000 - 23000 - 14600)

8759.380000000001

In [19]:
9689.4 / 210000

0.04614

In [24]:
(8759.38 + TaxCalculator.federal_income_tax(210000 - 23000 - 4150 - 14600)) / 210000

0.19262085714285712

In [172]:
fica_ = 168600 * 0.062 + 210000 * 0.0145

In [186]:
(210000 - fica_ - itemized_) / 12

13987.545833333332

In [180]:
210000 - fica_ - standard_

157251.41999999998

In [176]:
standard_ = \
TaxCalculator.tax_liability([0, 20000, 35000, 40000, 75000, 500000, 1000000],
                            [0.014, 0.0175, 0.035, 0.0525, 0.0637, 0.0897, 0.1075], 210000 - 23000 - 14600) \
+ TaxCalculator.federal_income_tax(210000 - 23000 - 4150 - 5000 - 14600)

In [177]:
itemized_ = \
TaxCalculator.tax_liability([0, 20000, 35000, 40000, 75000, 500000, 1000000],
                            [0.014, 0.0175, 0.035, 0.0525, 0.0637, 0.0897, 0.1075], 210000 - 23000 - 49500) \
+ TaxCalculator.federal_income_tax(210000 - 23000 - 4150 - 5000 - 49500)

In [36]:
standard_ - itemized_

10599.130000000005

In [11]:
mortgage_rate = 0.065

In [12]:
744000 * mortgage_rate / 12 * (1 + mortgage_rate / 12) ** (12 * 30) / ((1 + mortgage_rate / 12) ** (12 * 30) - 1)

4702.586094787662

In [13]:
_ * 360

1692930.9941235583

In [ ]:
R = .09

In [ ]:
45 * (1+R)**30 + sum()